#INIT

In [0]:
%sql
DROP TABLE IF EXISTS silver.crm_cust_info

In [0]:
%sql
CREATE TABLE silver.crm_cust_info (
  Customer_id INT,
  Customer_key VARCHAR(50),
  Firstname VARCHAR(50),
  Lastname VARCHAR(50),
  Marital_status VARCHAR(50),
  Gender VARCHAR(50),
  Create_date DATE,
  dwh_create_date TIMESTAMP
)

#READING FROM BRONZE

In [0]:
%sql
SELECT 
  cst_id,
  cst_key,
  cst_firstname,
  cst_lastname,
  cst_marital_status,
  cst_gndr,
  cst_create_date,
  current_timestamp()
FROM bronze.crm_cust_info

# DATA TRANSFORMATION

#Trimming

In [0]:
%sql
SELECT 
  cst_id,
  cst_key,
  TRIM(cst_firstname),
  TRIM(cst_lastname),
  TRIM(cst_marital_status),
  TRIM(cst_gndr),
  cst_create_date
FROM (
  SELECT *,
  ROW_NUMBER() OVER (PARTITION BY cst_key ORDER BY cst_create_date DESC)
  FROM bronze.crm_cust_info
  WHERE cst_id IS NOT NULL
  )t

#Normalization

In [0]:
%sql
SELECT 
  CASE WHEN UPPER(TRIM(cst_marital_status)) = 'S' THEN 'Single'
       WHEN UPPER(TRIM(cst_marital_status)) = 'M' THEN 'Married'
       ELSE 'n/a'
  END AS cst_marital_status,
  CASE WHEN UPPER(TRIM(cst_gndr)) = 'M' THEN 'Male'
       WHEN UPPER(TRIM(cst_gndr)) = 'F' THEN 'Female'
       ELSE 'n/a'
  END AS cst_gndr
FROM bronze.crm_cust_info

#RENAMING THE COLUMNS

In [0]:
%sql
SELECT 
  cst_id AS Customer_id,
  cst_key AS Customer_key,
  TRIM(cst_firstname) AS Firstname,
  TRIM(cst_lastname) AS Lastname,
  CASE WHEN UPPER(TRIM(cst_marital_status)) = 'S' THEN 'Single'
       WHEN UPPER(TRIM(cst_marital_status)) = 'M' THEN 'Married'
       ELSE 'n/a'
  END AS Marital_status,
  CASE WHEN UPPER(TRIM(cst_gndr)) = 'M' THEN 'Male'
       WHEN UPPER(TRIM(cst_gndr)) = 'F' THEN 'Female'
       ELSE 'n/a'
  END AS Gender,
  cst_create_date AS Create_date
FROM (
  SELECT *,
  ROW_NUMBER() OVER (PARTITION BY cst_key ORDER BY cst_create_date DESC) AS Flag_last
  FROM bronze.crm_cust_info
  WHERE cst_id IS NOT NULL
  )t WHERE Flag_last = 1

# WRITE INTO SILVER TABLE

In [0]:
%sql
INSERT INTO silver.crm_cust_info (
  Customer_id,
  Customer_key,
  Firstname,
  Lastname,
  Marital_status,
  Gender,
  Create_date,
  dwh_create_date
)
SELECT 
  cst_id AS Customer_id,
  cst_key AS Customer_key,
  TRIM(cst_firstname) AS Firstname,
  TRIM(cst_lastname) AS Lastname,
  CASE WHEN UPPER(TRIM(cst_marital_status)) = 'S' THEN 'Single'
       WHEN UPPER(TRIM(cst_marital_status)) = 'M' THEN 'Married'
       ELSE 'n/a'
  END AS Marital_status,
  CASE WHEN UPPER(TRIM(cst_gndr)) = 'M' THEN 'Male'
       WHEN UPPER(TRIM(cst_gndr)) = 'F' THEN 'Female'
       ELSE 'n/a'
  END AS Gender,
  cst_create_date AS Create_date,
  current_timestamp() AS dwh_create_date
FROM (
  SELECT *,
  ROW_NUMBER() OVER (PARTITION BY cst_key ORDER BY cst_create_date DESC) AS Flag_last
  FROM bronze.crm_cust_info
  WHERE cst_id IS NOT NULL
  )t WHERE Flag_last = 1

#TESTING

In [0]:
%sql
SELECT * FROM silver.crm_cust_info